In [ ]:
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput

role = get_execution_role()

In [ ]:
!aws s3 cp s3://redshift-downloads/redshift-ml/customer_activity/customer_activity.csv s3://20211117-redshiftml/redshift-ml/customer_churn/customer_activity.csv

In [ ]:
data= pd.read_csv('s3://20211117-redshiftml/redshift-ml/customer_churn/customer_activity.csv')
model_data = data[['Churn?', 'Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge']]
# model_data = data.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge', 'Record Date'], axis=1)
model_data = pd.get_dummies(model_data)
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)], axis=1)
model_data.head()

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

sagemaker_session = sagemaker.Session()
input_train = sagemaker_session.upload_data(path='train.csv', bucket='20211117-redshiftml', key_prefix='redshift-ml/customer_churn_train')
input_validation = sagemaker_session.upload_data(path='validation.csv', bucket='20211117-redshiftml', key_prefix='redshift-ml/customer_churn_validation')

content_type='text/csv'
s3_input_train = TrainingInput(input_train, content_type=content_type)
s3_input_validation = TrainingInput(input_validation, content_type=content_type)

In [ ]:
print('学習用データの格納先: ' ,input_train)

### XGBoost の学習

In [ ]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-1")
sess = sagemaker.Session()

hyperparameters = {"max_depth":"5",
                        "eta":"0.2",
                        "gamma":"4",
                        "min_child_weight":"6",
                        "subsample":"0.8",
                        "objective":"binary:logistic",
                        "num_round":"100"}

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    hyperparameters=hyperparameters,
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    sagemaker_session=sess)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

### 推論用エンドポイントの準備

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

SyntaxError: invalid syntax (<ipython-input-50-1d8c937e50d9>, line 1)

In [ ]:
ローカル推論

In [ ]:
CREATE MODEL customer_churn
FROM 'sagemaker-xgboost-2021-12-01-05-41-01-585'
FUNCTION customer_churn_predict (float, float, float, float)
RETURNS float
IAM_ROLE 'arn:aws:iam::815969174475:role/RedshiftML'
SETTINGS (S3_BUCKET '20211117-redshiftml');

In [ ]:
### リモート推論
モデルを作成する

In [ ]:
DROP MODEL IF EXISTS remote_customer_churn;
CREATE MODEL remote_customer_churn
FUNCTION remote_fn_customer_churn_predict (float, float, float, float)
RETURNS float
SAGEMAKER 'sagemaker-xgboost-2021-12-01-05-52-40-586'
IAM_ROLE 'arn:aws:iam::815969174475:role/RedshiftML';

推論をリクエストする

In [ ]:
DROP TABLE IF EXISTS customer_churn_xgb;
CREATE TABLE customer_churn_xgb (
churn int,
day_charge float, 
eve_charge float,
night_charge float,
intl_charge float);

COPY customer_churn_xgb
FROM 's3://20211117-redshiftml/redshift-ml/customer_churn_validation/'
IAM_ROLE 'arn:aws:iam::815969174475:role/RedshiftML' IGNOREHEADER 1 CSV;

DROP MODEL IF EXISTS remote_customer_churn;
CREATE MODEL remote_customer_churn
FUNCTION remote_fn_customer_churn_predict (float, float, float, float)
RETURNS float
SAGEMAKER 'sagemaker-xgboost-2021-12-01-05-52-40-586'
IAM_ROLE 'arn:aws:iam::815969174475:role/RedshiftML';

SELECT remote_fn_customer_churn_predict(
day_charge, 
eve_charge,
night_charge,
intl_charge);) AS churn_proba 
FROM customer_churn_xgb 

In [ ]:
select demo_ml.remote_fn_customer_churn_predict(
 customer_churn_xgb.day_charge, 
eve_charge,
night_charge,
intl_charge);) as churn_proba 
from demo_ml.customer_churn_xgb 